# Calling Tools

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. While the name implies that the model is performing some action, this is actually not the case! The model is merely coming up with the arguments to a tool, and actually running a [tool](/docs/modules/agents/tools/) (or not) is up to the user. For example, if you want to [extract output matching some schema](/docs/use_cases/extraction/) from unstructured text, you could give the model an "extraction" tool that takes parameters matching the desired schema, then treat the generated output as your final result.

Note that [not all chat models](/docs/integrations/chat/) support tool calling currently.

A [tool call object](https://api.js.langchain.com/types/langchain_core_messages_tool.ToolCall.html) includes a `name`, `arguments` option, and an optional `id`.

Many LLM providers, including [Anthropic](/docs/integrations/chat/anthropic), [Google Vertex](/docs/integrations/chat/google_vertex_ai),
[Mistral](/docs/integrations/chat/mistral), [OpenAI](/docs/integrations/chat/openai), and others, 
support variants of a tool calling feature. These features typically allow requests 
to the LLM to include available tools and their schemas, and for responses to include 
calls to these tools.

For instance, given a search engine tool, an LLM might handle a 
query by first calling the search engine tool by generated required parameters in the right format. The system calling the LLM can 
receive these generated parameters and use them to execute the tool, then the output to the LLM to inform its 
response. LangChain includes a suite of [built-in tools](/docs/integrations/tools/) 
and supports several methods for defining your own [custom tools](/docs/modules/tools/custom_tools). 
Tool-calling is extremely useful for building [tool-using chains and agents](/docs/use_cases/tool_use), 
and for getting structured outputs from models more generally.

Providers adopt different conventions for formatting tool schemas and tool calls. 
For instance, Anthropic returns tool calls as parsed structures within a larger content block:

```json
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]
```

whereas OpenAI separates tool calls into a distinct parameter, with arguments as JSON strings:

```json
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}
```

LangChain implements standard interfaces for defining tools, passing them to LLMs, 
and representing tool calls.

## Passing tools to LLMs

Chat models that support tool calling features implement a [`.bindTools()`](https://api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools) method, which 
receives a list of LangChain [tool objects](https://api.js.langchain.com/classes/langchain_core_tools.StructuredTool.html) 
and binds them to the chat model in its expected format. Subsequent invocations of the 
chat model will include tool schemas in its calls to the LLM.

Let's walk through a few examples. First, make sure you have required dependencies installed. We'll use Mistral by default, but you can use any [tool calling model](/docs/integrations/chat/):

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/core @langchain/mistralai
</Npm2Yarn>
```

```bash
MISTRAL_API_KEY=your-api-key
```

A number of models implement helper methods that will take care of formatting and binding different function-like objects to the model.
Let's take a look at how we might take the following Zod function schema and get different models to invoke it:

In [1]:
import { z } from "zod";

/**
 * Note that the descriptions here are crucial, as they will be passed along
 * to the model along with the class name.
 */
const calculatorSchema = z.object({
  operation: z
    .enum(["add", "subtract", "multiply", "divide"])
    .describe("The type of operation to execute."),
  number1: z.number().describe("The first number to operate on."),
  number2: z.number().describe("The second number to operate on."),
});

We can use the `ChatMistral.bindTools()` method to handle converting this LangChain tool to Mistral's format and binding it to the model (i.e., passing it in each time the model is invoked). Let's create a `DynamicStructuredTool` implementing the above logic, then bind it to the model:

In [2]:
import { ChatMistralAI } from "@langchain/mistralai";
import { DynamicStructuredTool } from "@langchain/core/tools";

const calculatorTool = new DynamicStructuredTool({
  name: "calculator",
  description: "Can perform mathematical operations.",
  schema: calculatorSchema,
  func: async ({ operation, number1, number2 }) => {
    // Functions must return strings
    if (operation === "add") {
      return `${number1 + number2}`;
    } else if (operation === "subtract") {
      return `${number1 - number2}`;
    } else if (operation === "multiply") {
      return `${number1 * number2}`;
    } else if (operation === "divide") {
      return `${number1 / number2}`;
    } else {
      throw new Error("Invalid operation.");
    }
  }
})

const llm = new ChatMistralAI({
  model: "mistral-large",
  temperature: 0,
});

const llmWithTools = llm.bindTools([calculatorTool]);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_cLBi7NjrehSEPoXr21i08NER",
          type: "function",
          function: [Object]
        }
      ]
    }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_cLBi7NjrehSEPoXr21i08NER",
        type: "function",
        function: {
          name: "calculator",
          arguments: '{"operation":"multiply","number1":3,"number2":12}'
        }
      }
    ]
  }
}

Now, let's invoke it! We expect the model to use the calculator to answer the question:

In [ ]:
const res = await llmWithTools.invoke("What is 3 * 12");

console.log(res.tool_calls);


> #### See the LangSmith trace [here](https://smith.langchain.com/public/cb36c6e5-e478-463c-801a-0c1b09ace04d/r).

We can see that the response message contains a `tool_calls` field when the model decides to call the tool. This will be in LangChain's shared format.

The `.tool_calls` attribute should contain valid tool calls. Note that on occasion, 
model providers may output malformed tool calls (e.g., arguments that are not 
valid JSON). When parsing fails in these cases, the message will contain instances of 
of [InvalidToolCall](https://api.js.langchain.com/types/langchain_core_messages_tool.InvalidToolCall.html) objects in the `.invalid_tool_calls` attribute. An `InvalidToolCall` can have 
a name, string arguments, identifier, and error message.

### Streaming

When tools are called in a streaming context, 
[message chunks](https://api.js.langchain.com/classes/langchain_core_messages.BaseMessageChunk.html) 
will be populated with [tool call chunk](https://api.js.langchain.com/types/langchain_core_messages_tool.ToolCallChunk.html)
objects in a list via the `.tool_call_chunks` attribute. A `ToolCallChunk` includes 
optional string fields for the tool `name`, `args`, and `id`, and includes an optional 
integer field `index` that can be used to join chunks together. Fields are optional 
because portions of a tool call may be streamed across different chunks (e.g., a chunk 
that includes a substring of the arguments may have null values for the tool name and id).

Because message chunks inherit from their parent message class, an 
[AIMessageChunk](https://api.js.langchain.com/classes/langchain_core_messages.AIMessageChunk.html) 
with tool call chunks will also include `.tool_calls` and `.invalid_tool_calls` fields. 
These fields are parsed best-effort from the message's tool call chunks.

Note that not all providers currently support streaming for tool calls, and will yield a single chunk with the entire call when `.stream()` is called on them.

Example:

Note that adding message chunks will merge their corresponding tool call chunks. This is the principle by which LangChain's various [tool output parsers](/docs/modules/model_io/output_parsers/types/openai_tools/) support streaming.

For example, below we accumulate tool call chunks:

{
  type: "function",
  function: {
    name: "calculator",
    description: "A simple calculator tool",
    parameters: {
      type: "object",
      properties: {
        operation: {
          type: "string",
          enum: [Array],
          description: "The type of operation to execute."
        },
        number1: {
          type: "number",
          description: "The first number to operate on."
        },
        number2: {
          type: "number",
          description: "The second number to operate on."
        }
      },
      required: [ "operation", "number1", "number2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

## Next steps

- **Output parsing**: See [OpenAI Tools output parsers](/docs/modules/model_io/output_parsers/types/openai_tools) to learn about extracting the function calling API responses into various formats.
- **Structured output chains**: Some models have constructors that handle creating a structured output chain for you ([OpenAI](/docs/integrations/chat/openai#withstructuredoutput--), [Mistral](/docs/integrations/chat/mistral#withstructuredoutput--)).
- **Tool use**: See how to construct chains and agents that actually call the invoked tools in [these guides](/docs/use_cases/tool_use/).
